In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

K=torch.rand(2,3,3)
K,K.transpose(-2, -1),torch.matmul(K, K.transpose(-2, -1))

(tensor([[[0.3116, 0.4082, 0.3891],
          [0.6148, 0.3447, 0.5520],
          [0.9656, 0.9069, 0.3881]],
 
         [[0.5146, 0.1651, 0.6395],
          [0.2647, 0.2229, 0.2476],
          [0.6552, 0.2878, 0.7888]]]),
 tensor([[[0.3116, 0.6148, 0.9656],
          [0.4082, 0.3447, 0.9069],
          [0.3891, 0.5520, 0.3881]],
 
         [[0.5146, 0.2647, 0.6552],
          [0.1651, 0.2229, 0.2878],
          [0.6395, 0.2476, 0.7888]]]),
 tensor([[[0.4152, 0.5471, 0.8221],
          [0.5471, 0.8015, 1.1204],
          [0.8221, 1.1204, 1.9054]],
 
         [[0.7011, 0.3314, 0.8891],
          [0.3314, 0.1811, 0.4329],
          [0.8891, 0.4329, 1.1343]]]))

In [2]:
src_vocab_size=5 # 三个词
max_seq_length=3 # 三个词语
import torch
from torch.nn.utils.rnn import pad_sequence

# 输入数据
src_data = [['你', '是', '谁'], ['谢', '谢']]

# 将每个序列转换为索引表示
vocab = {'<PAD>':0,'你': 1, '是': 2, '谁': 3, '谢': 4}
src_data_idx = [[vocab[word] for word in seq] for seq in src_data]

# 转换为 PyTorch 的 Tensor
src_tensors = [torch.tensor(seq) for seq in src_data_idx]

# 进行填充
padded_src_data = pad_sequence(src_tensors, batch_first=True, padding_value=vocab['<PAD>'])

eb=nn.Embedding(src_vocab_size,3)
tgt_data=eb(padded_src_data)

tgt_data

tensor([[[-0.8392, -0.2234, -1.6956],
         [ 0.5203, -1.0467, -0.5132],
         [ 0.5145,  1.5801, -0.4507]],

        [[ 1.9833, -1.1405, -0.2046],
         [ 1.9833, -1.1405, -0.2046],
         [ 0.4353,  0.7404, -0.9633]]], grad_fn=<EmbeddingBackward0>)

In [3]:
#src(batch_size, seq_length)
tgt=padded_src_data
seq_length=tgt.size(1)
tgt_tmp=(tgt!=0).unsqueeze(1).unsqueeze(3)
nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool() #将主对角线及其以上的元素设为零，主对角线以下的元素设为 1
mask=tgt_tmp&nopeak_mask
tgt,nopeak_mask,tgt_tmp,mask

(tensor([[1, 2, 3],
         [4, 4, 0]]),
 tensor([[[ True, False, False],
          [ True,  True, False],
          [ True,  True,  True]]]),
 tensor([[[[ True],
           [ True],
           [ True]]],
 
 
         [[[ True],
           [ True],
           [False]]]]),
 tensor([[[[ True, False, False],
           [ True,  True, False],
           [ True,  True,  True]]],
 
 
         [[[ True, False, False],
           [ True,  True, False],
           [False, False, False]]]]))

In [4]:
x=tgt_data
batch_size, seq_length, d_model = x.size() # 加一个头
tgt_data=x.view(batch_size, seq_length, 1, d_model).transpose(1, 2)
attn_scores = torch.matmul(tgt_data, tgt_data.transpose(-2, -1))
attn_scores=attn_scores.masked_fill(mask == 0, -1e9)
attn_scores

tensor([[[[ 3.6293e+00, -1.0000e+09, -1.0000e+09],
          [ 6.6746e-01,  1.6296e+00, -1.0000e+09],
          [-2.0676e-02, -1.1548e+00,  2.9645e+00]]],


        [[[ 5.2761e+00, -1.0000e+09, -1.0000e+09],
          [ 5.2761e+00,  5.2761e+00, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09]]]],
       grad_fn=<MaskedFillBackward0>)